<a href="https://colab.research.google.com/github/ShixuanGuo/Airbnb-Data-Analysis/blob/master/XGboostModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [3]:
data = pd.read_csv("/content/drive/My Drive/finch research/without_desc.csv")

In [5]:
data.head()

,grade,home_ownership,pymnt_plan,purpose,addr_state,initial_list_status,application_type,disbursement_method,emp_length,loan_amnt,int_rate,annual_inc,dti,delinq_2yrs,fico_range_low,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,last_fico_range_high,last_fico_range_low,mths_since_last_major_derog,open_acc_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,inq_fi,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,year_diff,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,loan_status
0,C,MORTGAGE,n,debt_consolidation,PA,w,Individual,Cash,11.0,3600.0,13.99,55000.0,5.91,0.0,675.0,1.0,30.0,0.0,7.0,0.0,2765.0,29.7,564.0,560.0,30.0,2.0,0.0,1.0,21.0,36.0,3.0,3.0,722.0,34.0,3.0,4.0,4.0,20701.0,1506.0,37.2,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,17.0,0.0,3.0,76.9,0.0,0.0,0
1,C,MORTGAGE,n,small_business,SD,w,Individual,Cash,11.0,24700.0,11.99,65000.0,16.06,1.0,715.0,4.0,6.0,0.0,22.0,0.0,21470.0,19.2,699.0,695.0,44.0,1.0,0.0,1.0,19.0,73.0,2.0,3.0,6472.0,29.0,0.0,6.0,4.0,9733.0,57830.0,27.1,113.0,192.0,2.0,2.0,4.0,2.0,0.0,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,21.0,0.0,2.0,97.4,7.7,0.0,0
2,B,MORTGAGE,n,home_improvement,IL,w,Joint App,Cash,11.0,20000.0,10.78,63000.0,10.78,0.0,695.0,0.0,0.0,0.0,6.0,0.0,7869.0,56.2,704.0,700.0,44.0,0.0,0.0,4.0,19.0,73.0,0.0,2.0,2081.0,65.0,2.0,1.0,6.0,31617.0,2737.0,55.9,125.0,184.0,14.0,14.0,5.0,101.0,0.0,10.0,0.0,0.0,2.0,3.0,2.0,4.0,6.0,4.0,7.0,20.0,0.0,0.0,100.0,50.0,0.0,0
3,C,MORTGAGE,n,debt_consolidation,NJ,w,Individual,Cash,11.0,35000.0,14.85,110000.0,17.06,0.0,785.0,0.0,0.0,0.0,13.0,0.0,7802.0,11.6,679.0,675.0,44.0,1.0,0.0,1.0,23.0,70.0,1.0,1.0,6987.0,45.0,0.0,0.0,2.0,23192.0,54962.0,12.1,36.0,87.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,4.0,5.0,8.0,10.0,2.0,10.0,13.0,12.0,0.0,1.0,100.0,0.0,0.0,0
4,F,MORTGAGE,n,major_purchase,PA,w,Individual,Cash,3.0,10400.0,22.45,104433.0,25.37,1.0,695.0,3.0,12.0,0.0,12.0,0.0,21929.0,64.5,704.0,700.0,44.0,1.0,0.0,3.0,14.0,84.0,4.0,7.0,9702.0,78.0,2.0,3.0,10.0,27644.0,4567.0,77.5,128.0,210.0,4.0,4.0,6.0,4.0,12.0,1.0,12.0,0.0,4.0,6.0,5.0,9.0,10.0,7.0,19.0,22.0,0.0,4.0,96.6,60.0,0.0,0


In [ ]:
data.shape

(2260668, 64)

###Split train and test dataset

In [4]:
onehot_columns = ['home_ownership', 'purpose', 'addr_state', 'initial_list_status', 'application_type', 'disbursement_method','grade', 'pymnt_plan']
onehot_df = data[onehot_columns]
onehot_df = pd.get_dummies(onehot_df, columns = onehot_columns)
score_onehot_drop = data.drop(onehot_columns, axis = 1)
data_encode = pd.concat([score_onehot_drop, onehot_df], axis = 1)

In [5]:
y=data['loan_status']
x=data_encode.loc[:, data_encode.columns != 'loan_status']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=2)

In [10]:
x.head()

,grade,pymnt_plan,emp_length,loan_amnt,int_rate,annual_inc,dti,delinq_2yrs,fico_range_low,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,last_fico_range_high,last_fico_range_low,mths_since_last_major_derog,open_acc_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,inq_fi,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,...,addr_state_KY,addr_state_LA,addr_state_MA,addr_state_MD,addr_state_ME,addr_state_MI,addr_state_MN,addr_state_MO,addr_state_MS,addr_state_MT,addr_state_NC,addr_state_ND,addr_state_NE,addr_state_NH,addr_state_NJ,addr_state_NM,addr_state_NV,addr_state_NY,addr_state_OH,addr_state_OK,addr_state_OR,addr_state_PA,addr_state_RI,addr_state_SC,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,disbursement_method_Cash,disbursement_method_DirectPay
0,C,n,11.0,3600.0,13.99,55000.0,5.91,0.0,675.0,1.0,30.0,0.0,7.0,0.0,2765.0,29.7,564.0,560.0,30.0,2.0,0.0,1.0,21.0,36.0,3.0,3.0,722.0,34.0,3.0,4.0,4.0,20701.0,1506.0,37.2,148.0,128.0,3.0,3.0,1.0,4.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0
1,C,n,11.0,24700.0,11.99,65000.0,16.06,1.0,715.0,4.0,6.0,0.0,22.0,0.0,21470.0,19.2,699.0,695.0,44.0,1.0,0.0,1.0,19.0,73.0,2.0,3.0,6472.0,29.0,0.0,6.0,4.0,9733.0,57830.0,27.1,113.0,192.0,2.0,2.0,4.0,2.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0
2,B,n,11.0,20000.0,10.78,63000.0,10.78,0.0,695.0,0.0,0.0,0.0,6.0,0.0,7869.0,56.2,704.0,700.0,44.0,0.0,0.0,4.0,19.0,73.0,0.0,2.0,2081.0,65.0,2.0,1.0,6.0,31617.0,2737.0,55.9,125.0,184.0,14.0,14.0,5.0,101.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0
3,C,n,11.0,35000.0,14.85,110000.0,17.06,0.0,785.0,0.0,0.0,0.0,13.0,0.0,7802.0,11.6,679.0,675.0,44.0,1.0,0.0,1.0,23.0,70.0,1.0,1.0,6987.0,45.0,0.0,0.0,2.0,23192.0,54962.0,12.1,36.0,87.0,2.0,2.0,1.0,2.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0
4,F,n,3.0,10400.0,22.45,104433.0,25.37,1.0,695.0,3.0,12.0,0.0,12.0,0.0,21929.0,64.5,704.0,700.0,44.0,1.0,0.0,3.0,14.0,84.0,4.0,7.0,9702.0,78.0,2.0,3.0,10.0,27644.0,4567.0,77.5,128.0,210.0,4.0,4.0,6.0,4.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0


In [ ]:
# not use
onehot_columns = ['home_ownership', 'purpose', 'addr_state', 'initial_list_status', 'application_type', 'disbursement_method']
onehot_df = score_df[onehot_columns]
onehot_df = pd.get_dummies(onehot_df, columns = onehot_columns)
score_onehot_drop = score_df.drop(onehot_columns, axis = 1)
score_onehot = pd.concat([score_onehot_drop, onehot_df], axis = 1)

### Hyperparameter



1.   Learning rate



In [ ]:
k_list=[]
for kkk in np.arange(0.05, 0.30, 0.01):
    xg=XGBClassifier(learning_rate=kkk,
                     n_estimators=100,
                     colsample_bytree = 0.4,
                     subsample = 0.8, 
                     max_depth=7,
                     gamma=10)
    xg.fit(x_train,y_train)
    y_hat=xg.predict(x_test)
    accuracy = accuracy_score(y_test, y_hat)
    k_list.append(accuracy)

In [ ]:
index=k_list.index(min(k_list))
grif=np.linspace(0.05, 0.30, num=25)[index]
print(grif)


2.   Number of estimators



In [ ]:
n_list=[]
for kkk in range(100, 1000,10):
    xg=XGBClassifier(learning_rate=grif,
                     n_estimators=kkk,
                     colsample_bytree = 0.4,
                     subsample = 0.8,
                     objective='binary:logistic', 
                     max_depth=7,
                     gamma=10)
    xg.fit(x_train,y_train)
    y_hat=xg.predict(x_test)
    accuracy = accuracy_score(y_test, y_hat)
    n_list.append(accuracy)

In [ ]:
index=n_list.index(min(n_list))
grif2=range(100, 1000,10)[index]
print(grif2)
print(min(n_list))

3. Subsample (0.8-1)

In [ ]:
s_list=[]
for kkk in np.linspace(0.8, 1, num=10):
    xg=XGBClassifier(learning_rate=grif,
                     n_estimators=grif2,
                     colsample_bytree = 0.4,
                     subsample = kkk,
                     objective='binary:logistic', 
                     max_depth=7,
                     gamma=10)
    xg.fit(x_train,y_train)
    y_hat=xg.predict(x_test)
    accuracy = accuracy_score(y_test, y_hat)
    s_list.append(accuracy)

In [ ]:
index=s_list.index(min(s_list))
grif3=np.linspace(0.8, 1, num=10)[index]
print(grif3)
print(min(s_list))

4. Colsample-bytree (0.3-0.8)

In [ ]:
c_list=[]
for kkk in range(0.3,0.8,num=25):
    xg=XGBClassifier(learning_rate=grif,
                     n_estimators=grif2,
                     colsample_bytree = kkk,
                     subsample = grif3,
                     objective='binary:logistic', 
                     max_depth=7,
                     gamma=10)
    xg.fit(x_train,y_train)
    y_hat=xg.predict(x_test)
    accuracy = accuracy_score(y_test, y_hat)
    c_list.append(accuracy)

In [ ]:
index=c_list.index(min(c_list))
grif4=range(0.3,0.8,num=25)[index]
print(grif4)
print(min(c_list))

### Fit Model

In [ ]:
xg=XGBClassifier(learning_rate=grif,
                     n_estimators=grif2,
                     colsample_bytree = grif4,
                     subsample = grif3,
                     objective='binary:logistic', 
                     max_depth=7,
                     gamma=10)
xg.fit(x_train,y_train)
y_hat=xg.predict(x_test)
accuray = accuracy_score(y_test, y_hat)
print(accuracy)